<a href="https://colab.research.google.com/github/Nacho2904/orga_de_datos/blob/main/tp_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TP1 - Pandas y Visualización de datos

## SetUp

In [84]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from google.colab import drive 

drive.mount('/content/gdrive')
path_a_cast = 'gdrive/MyDrive/Movie dataset/cast.csv'
path_a_companies = 'gdrive/MyDrive/Movie dataset/companies.csv'
path_a_crew = 'gdrive/MyDrive/Movie dataset/crew.csv'
path_a_actores = 'gdrive/MyDrive/Movie dataset/imdb_actors.csv'
path_a_keywords = 'gdrive/MyDrive/Movie dataset/keywords.csv'
path_a_links = 'gdrive/MyDrive/Movie dataset/links.csv'
path_a_movies = 'gdrive/MyDrive/Movie dataset/movies.csv'
path_a_ratings = 'gdrive/MyDrive/Movie dataset/ratings.csv'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


## Parte I: Pandas

### P2

- De las 10 películas más populares, ¿cuál es el título de la película que tiene más géneros asociados? ¿Cuáles son esos géneros?

In [85]:
# Leemos el dataset de películas con Pandas
df_movies = pd.read_csv(path_a_movies)
df_movies.head(1)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,Toy Story Collection,30000000,"Animation,Comedy,Family",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0


In [86]:
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 43021 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   33585 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

Vemos que el dataset soluciona nuestra primera cuestión: cómo ordenar las películas por popularidad. Examinamos la columna **popularity** que nos viene por defecto.

In [87]:
df_movies["popularity"]

0        21.946943
1        17.015539
2          11.7129
3         3.859495
4         8.387519
           ...    
45461     0.072051
45462     0.178241
45463     0.903007
45464     0.003503
45465     0.163015
Name: popularity, Length: 45466, dtype: object

Notamos más arriba del .info que su tipo es *object*. Intentamos transformarla a *float* para ver si hay valores que no son numéricos (si surge una excepción, habrá valores no numéricos).

In [88]:
#comento todo esto incluido el error porque si tengo que correr todo de vuelta me para la ejecución
#popularity_as_float = df_movies["popularity"].map(float)
"""
---------------------------------------------------------------------------

ValueError                                Traceback (most recent call last)

<ipython-input-21-384c6448775b> in <module>
----> 1 popularity_as_float = df_movies["popularity"].map(float)

2 frames

/usr/local/lib/python3.7/dist-packages/pandas/core/series.py in map(self, arg, na_action)
   4159         dtype: object
   4160         
-> 4161         new_values = super()._map_values(arg, na_action=na_action)
   4162         return self._constructor(new_values, index=self.index).__finalize__(
   4163             self, method="map"

/usr/local/lib/python3.7/dist-packages/pandas/core/base.py in _map_values(self, mapper, na_action)
    868 
    869         # mapper is a function
--> 870         new_values = map_f(values, mapper)
    871 
    872         return new_values

/usr/local/lib/python3.7/dist-packages/pandas/_libs/lib.pyx in pandas._libs.lib.map_infer()

ValueError: could not convert string to float: 'Beware Of Frost Bites'


"""

'\n---------------------------------------------------------------------------\n\nValueError                                Traceback (most recent call last)\n\n<ipython-input-21-384c6448775b> in <module>\n----> 1 popularity_as_float = df_movies["popularity"].map(float)\n\n2 frames\n\n/usr/local/lib/python3.7/dist-packages/pandas/core/series.py in map(self, arg, na_action)\n   4159         dtype: object\n   4160         \n-> 4161         new_values = super()._map_values(arg, na_action=na_action)\n   4162         return self._constructor(new_values, index=self.index).__finalize__(\n   4163             self, method="map"\n\n/usr/local/lib/python3.7/dist-packages/pandas/core/base.py in _map_values(self, mapper, na_action)\n    868 \n    869         # mapper is a function\n--> 870         new_values = map_f(values, mapper)\n    871 \n    872         return new_values\n\n/usr/local/lib/python3.7/dist-packages/pandas/_libs/lib.pyx in pandas._libs.lib.map_infer()\n\nValueError: could not conv

Efectivamente, hay filas en las que la columna popularidad contiene valores no numéricos. Abajo mostramos la que encontramos al intentar aplicar el map.

In [89]:
df_movies[df_movies["popularity"] == 'Beware Of Frost Bites']["popularity"].iloc[0]

'Beware Of Frost Bites'

Esto nos importa porque mientras haya filas que posean esa cualidad, no podemos ordenar el dataset para obtener las 10 películas más populares. Contamos la cantidad de películas con este problema:

In [90]:
pelis_polemicas = pd.to_numeric(df_movies["popularity"], errors = 'coerce')
df_movies[pelis_polemicas.isnull()][["original_title","vote_count"]]

,original_title,vote_count
19729,Midnight Man,NaN
19730,"[{'iso_639_1': 'en', 'name': 'English'}]",NaN
29502,マルドゥック・スクランブル 排気,NaN
29503,"[{'iso_639_1': 'ja', 'name': '日本語'}]",NaN
35586,Avalanche Sharks,NaN
35587,"[{'iso_639_1': 'en', 'name': 'English'}]",NaN


De casi 45500 filas, solo 6 tienen problemas (y ninguna de ellas parece honestamente muy relevante, pues ninguna tiene ni un solo voto). Las ignoraremos.

In [91]:
df_movies_modificado = pd.DataFrame.copy(df_movies)
df_movies_modificado["popularity"] = pd.to_numeric(df_movies["popularity"], errors = 'coerce').fillna(value=0)
df_movies_ordenado_por_popularidad = df_movies_modificado.sort_values(by="popularity", ascending = False)

Nos quedamos con las columnas relevantes de la tabla obtenida

In [92]:
top_10_peliculas = df_movies_ordenado_por_popularidad.head(10)[["original_title","genres"]]
top_10_peliculas

,original_title,genres
30700,Minions,"Family,Animation,Adventure,Comedy"
33356,Wonder Woman,"Action,Adventure,Fantasy"
42222,Beauty and the Beast,"Family,Fantasy,Romance"
43644,Baby Driver,"Action,Crime"
24455,Big Hero 6,"Adventure,Family,Animation,Action,Comedy"
26564,Deadpool,"Action,Adventure,Comedy"
26566,Guardians of the Galaxy Vol. 2,"Action,Adventure,Comedy,Science Fiction"
14551,Avatar,"Action,Adventure,Fantasy,Science Fiction"
24351,John Wick,"Action,Thriller"
23675,Gone Girl,"Mystery,Thriller,Drama"


Finalmente, obtenemos la respuesta al ejercicio aplicando las transformaciones adecuadas a nuestra tabla.

In [93]:
cantidad_de_generos = lambda generos: len(generos.split(","))
top_10_peliculas["cantidad_de_generos"] = top_10_peliculas["genres"].map(cantidad_de_generos)
max_generos = max(top_10_peliculas["cantidad_de_generos"])
top_10_peliculas[top_10_peliculas["cantidad_de_generos"] == max_generos][["original_title","genres"]]

,original_title,genres
24455,Big Hero 6,"Adventure,Family,Animation,Action,Comedy"


### P27

- En todo el mundo el estado financia parcialmente al cine de las productoras locales. Una preocupación creciente es la existencia de ñoquis en las crew de las películas que son amigos o conocidos de la producción de las mismas a expensas de fondos del estado. La consultora “Espumadera” es una startup Argentina se dedica a encontrar posibles ñoquis para auditar en distintos contextos y tiene como clientes a organismos estatales de todo el mundo. Deseamos buscar anomalías para auditar en la crew de las películas, ¿Quienes superan el **percentil 95** de producciones en las que **participaron en roles distintos de directores, productores o escritores en el mismo año de estreno**?

In [94]:
df_crew = pd.read_csv(path_a_crew)
df_crew.info( )

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 464314 entries, 0 to 464313
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   id          464314 non-null  int64 
 1   crew_id     464314 non-null  int64 
 2   department  464314 non-null  object
 3   job         464314 non-null  object
 4   name        464314 non-null  object
dtypes: int64(2), object(3)
memory usage: 17.7+ MB


Teniendo la tabla *df_crew*, filtramos las columnas para quedarnos con las relevantes al análisis, y filtramos los roles a lo que nos pide el ejercicio.

In [95]:
df_crew_filtrado = df_crew[["id","crew_id","job","name"]]
df_crew_filtrado = df_crew_filtrado[df_crew_filtrado["job"].isin(["Director", "Producer", "Writer"])]

Ahora queremos agregar algo de información sobre las películas, específicamente su año de estreno. Para lograr eso podemos utilizar un join. Primero miramos un poco la tabla de películas

In [96]:
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 43021 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   33585 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

Nos interesa el *id* para poder utilizarlo en el *join*, y el *release_date* porque es casi exactamente lo que estamos buscando. También nos interesa conocer si toda la columna *release_date* está en formato correcto.

In [97]:
df_movies_filtrado = df_movies[["id", "release_date"]]
df_movies_filtrado["release_date"] 

0        1995-10-30
1        1995-12-15
2        1995-12-22
3        1995-12-22
4        1995-02-10
            ...    
45461           NaN
45462    2011-11-17
45463    2003-08-01
45464    1917-10-21
45465    2017-06-09
Name: release_date, Length: 45466, dtype: object

Solo con ver la columna a ojo vemos que hay un *NaN*, lo que significa que no todas las pelis tienen la fecha de estreno. Lidiaremos con eso utilizando un *inner join* para quedarnos con todas las pelis que sí tengan dicha columna. Dicho eso, filtramos otra vez nuestra tabla de películas para eliminar dichos casos, y nos quedamos con el año en cada fila.

In [98]:
extract_year = lambda date: date.split("-")[0]
df_movies_filtrado_not_null = pd.DataFrame.copy(df_movies_filtrado[df_movies_filtrado["release_date"].notnull()])
df_movies_with_year = pd.DataFrame.copy(df_movies_filtrado_not_null)
df_movies_with_year["release_year"] = df_movies_filtrado_not_null["release_date"].map(extract_year)
df_movies_with_year["release_year"].unique()

array(['1995', '1996', '1994', '1997', '1976', '1992', '1967', '1993',
       '1964', '1977', '1965', '1982', '1985', '1990', '1991', '1989',
       '1937', '1940', '1969', '1981', '1974', '1970', '1960', '1955',
       '1959', '1968', '1980', '1988', '1975', '2002', '1948', '1943',
       '1950', '1987', '1973', '1956', '1958', '1949', '1972', '1953',
       '1998', '1933', '2010', '1952', '1951', '1957', '1961', '1954',
       '1934', '1944', '1963', '1942', '1941', '1939', '1947', '1946',
       '1945', '1938', '1935', '1936', '1926', '1932', '1979', '1971',
       '1986', '2013', '1978', '1966', '1962', '1983', '1984', '1931',
       '1922', '1999', '1927', '1929', '1930', '1928', '2012', '1925',
       '2000', '1919', '1923', '1920', '1918', '1921', '2001', '2011',
       '1924', '2003', '2004', '1915', '1916', '1917', '2005', '2006',
       '1902', '1903', '2007', '2008', '2009', '1914', '1912', '1913',
       '1898', '1899', '1894', '1909', '1910', '1901', '1893', '1896',
      

Vemos que hay un par de errores, específicamente tenemos años 1, 12, 22.

In [99]:
df_movies_filtrado_not_null[df_movies_with_year["release_year"] == "1"]

,id,release_date
19730,1997-08-20,1


In [100]:
df_movies_filtrado_not_null[df_movies_with_year["release_year"] == "12"]

,id,release_date
29503,2012-09-29,12


In [101]:
df_movies_filtrado_not_null[df_movies_with_year["release_year"] == "22"]

,id,release_date
35587,2014-01-01,22


En los 3 casos notamos que el id está donde debería estar el release date y viceversa. Arreglamos esto.

In [102]:
df_movies_filtrado_not_null.at[df_movies_with_year["release_year"] == "1","release_date"] = "1997-08-20"
df_movies_filtrado_not_null.at[df_movies_with_year["id"] == "1997-08-20","id"] = 1
df_movies_filtrado_not_null.at[df_movies_with_year["release_year"] == "12","release_date"] = "2012-09-29"
df_movies_filtrado_not_null.at[df_movies_with_year["id"] == "2012-09-29","id"] = 12
df_movies_filtrado_not_null.at[df_movies_with_year["release_year"] == "22","release_date"] = "2014-01-01"
df_movies_filtrado_not_null.at[df_movies_with_year["id"] == "2014-01-01","id"] = 22

In [103]:
df_movies_with_year = pd.DataFrame.copy(df_movies_filtrado_not_null)
df_movies_with_year["release_year"] = df_movies_filtrado_not_null["release_date"].map(extract_year)
df_movies_with_year["release_year"].unique()

array(['1995', '1996', '1994', '1997', '1976', '1992', '1967', '1993',
       '1964', '1977', '1965', '1982', '1985', '1990', '1991', '1989',
       '1937', '1940', '1969', '1981', '1974', '1970', '1960', '1955',
       '1959', '1968', '1980', '1988', '1975', '2002', '1948', '1943',
       '1950', '1987', '1973', '1956', '1958', '1949', '1972', '1953',
       '1998', '1933', '2010', '1952', '1951', '1957', '1961', '1954',
       '1934', '1944', '1963', '1942', '1941', '1939', '1947', '1946',
       '1945', '1938', '1935', '1936', '1926', '1932', '1979', '1971',
       '1986', '2013', '1978', '1966', '1962', '1983', '1984', '1931',
       '1922', '1999', '1927', '1929', '1930', '1928', '2012', '1925',
       '2000', '1919', '1923', '1920', '1918', '1921', '2001', '2011',
       '1924', '2003', '2004', '1915', '1916', '1917', '2005', '2006',
       '1902', '1903', '2007', '2008', '2009', '1914', '1912', '1913',
       '1898', '1899', '1894', '1909', '1910', '1901', '1893', '1896',
      

Notamos también que antes la columna id era de tipo *object* por la presencia de las fechas mezcladas. Habiendo arreglado eso podemos arreglar todo a int

In [104]:
df_movies_with_year["id"] = pd.to_numeric(df_movies_with_year["id"], downcast="signed")
df_movies_with_year.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45379 entries, 0 to 45465
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            45379 non-null  int32 
 1   release_date  45379 non-null  object
 2   release_year  45379 non-null  object
dtypes: int32(1), object(2)
memory usage: 2.2+ MB


Por fin estamos en condiciones como para realizar el join. 

In [105]:
df_crew_with_years = pd.merge(df_crew_filtrado,df_movies_with_year,on='id', how = "inner")
df_crew_with_years

,id,crew_id,job,name,release_date,release_year
0,862,7879,Director,John Lasseter,1995-10-30,1995
1,862,12894,Producer,Bonnie Arnold,1995-10-30,1995
2,862,12896,Producer,Ralph Guggenheim,1995-10-30,1995
3,8844,4945,Director,Joe Johnston,1995-12-15,1995
4,8844,9184,Producer,Scott Kroopf,1995-12-15,1995
...,...,...,...,...,...,...
123161,111109,1051381,Writer,Lav Diaz,2011-11-17,2011
123162,67758,67753,Director,Mark L. Lester,2003-08-01,2003
123163,227506,1085341,Director,Yakov Protazanov,1917-10-21,1917
123164,227506,1195656,Producer,Joseph N. Ermolieff,1917-10-21,1917


Agrupamos por actor y por año de estreno:

In [106]:
df_crew_count_by_year = df_crew_with_years.groupby(["crew_id", "release_year"]).agg(
    appearances = ("release_year", "count")).reset_index()
df_crew_count_by_year.head()

,crew_id,release_year,appearances
0,1,1967,2
1,1,1971,1
2,1,1973,1
3,1,1977,2
4,1,1985,1


In [107]:
#volvemos a agrupar por actor para sacar la máxima cantidad de participaciones que logró en un solo año
df_max_crew_appearances_per_year = df_crew_count_by_year.groupby("crew_id").agg(max_appearances=("appearances",max)).reset_index()
quantile_95th = df_max_crew_appearances_per_year["max_appearances"].quantile(0.95)
df_max_crew_appearances_per_year[df_max_crew_appearances_per_year["max_appearances"] > quantile_95th]

,crew_id,max_appearances
9,24,4
15,32,4
19,40,4
22,59,7
28,67,4
...,...,...
39415,1341689,4
40582,1386894,6
41011,1410876,4
44669,1621038,4


Vemis que hay un total de 3371 miembros de una crew que superan dicho percentil. Podemos encontrar sus nombres fácilmente:

In [108]:
suspect_crew_members = pd.merge(df_crew_filtrado,
                                df_max_crew_appearances_per_year[df_max_crew_appearances_per_year["max_appearances"] > quantile_95th],
                                on='crew_id', 
                                how = "right")
suspect_crew_members


,id,crew_id,job,name,max_appearances
0,13,24,Director,Robert Zemeckis,4
1,105,24,Director,Robert Zemeckis,4
2,105,24,Writer,Robert Zemeckis,4
3,686,24,Director,Robert Zemeckis,4
4,686,24,Producer,Robert Zemeckis,4
...,...,...,...,...,...
13496,429779,1827779,Director,Shannon Hartman,4
13497,429779,1827779,Producer,Shannon Hartman,4
13498,337405,1827779,Director,Shannon Hartman,4
13499,419037,1827779,Director,Shannon Hartman,4


Por curiosidad, busco el top 5

In [109]:
really_suspect_crew_members = pd.merge(df_crew_filtrado,
                                df_max_crew_appearances_per_year.sort_values("max_appearances", ascending=False).head(5),
                                on='crew_id', 
                                how = "right")
really_suspect_crew_members["name"].unique()

array(['Charlie Chaplin', 'JP Siili', 'David Michael Latt',
       'Buster Keaton', 'Max Linder'], dtype=object)

### P20

- ¿Cuál es la probabilidad de que una película de un género en particular tenga una calificación promedio mayor a 7? Obtenga las probabilidades de cada género posible.

In [110]:
df_ratings = pd.read_csv(path_a_ratings)
df_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26024289 entries, 0 to 26024288
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int64  
 1   movieId    int64  
 2   rating     float64
 3   timestamp  int64  
dtypes: float64(1), int64(3)
memory usage: 794.2 MB


In [111]:
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 43021 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   33585 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

Filtramos las columnas importantes: Vamos a necesitar el id y el género para hacer un *join* con la tabla **ratings** y así tener también el nombre de la película. Primero que nada, sería útil conocer que géneros tenemos, y sacarnos de encima las pelis de las que no conocemos el género.

In [112]:
df_movies_filtered = df_movies[["id", "genres"]]
def add_genre(genre_string:str, known_genres:dict)->None:
  genres_in_string = genre_string.split(",")
  for genre in genres_in_string:
    if not known_genres.get(genre):
      known_genres.update({genre:len(known_genres)})

known_genres = {}
obtain_existing_genres = lambda genre: add_genre(genre, known_genres)
df_movies_filtered = df_movies_filtered[df_movies_filtered["genres"].notnull()]
df_movies_filtered["genres"].map(obtain_existing_genres)
df_movies_filtered["id"] = pd.to_numeric(df_movies_filtered["id"], errors = "coerce")
known_genres = sorted(list(known_genres.keys()))
known_genres

['Action',
 'Adventure',
 'Animation',
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Family',
 'Fantasy',
 'Foreign',
 'History',
 'Horror',
 'Music',
 'Mystery',
 'Romance',
 'Science Fiction',
 'TV Movie',
 'Thriller',
 'War',
 'Western']

Realizamos el join de **df_ratings** con **df_movies_filtered**, previamente filtrando las columnas relevantes del primero (movie_id, rating)..

In [113]:
df_ratings_filtered = df_ratings[["movieId", "rating"]]
df_ratings_filtered.columns = ["id", "rating"]
df_ratings_and_genres = pd.merge(df_ratings_filtered, df_movies_filtered, on="id", how="inner")
df_ratings_and_genres

,id,rating,genres
0,110,1.0,"Drama,Mystery,Romance"
1,110,3.5,"Drama,Mystery,Romance"
2,110,5.0,"Drama,Mystery,Romance"
3,110,5.0,"Drama,Mystery,Romance"
4,110,3.0,"Drama,Mystery,Romance"
...,...,...,...
11382416,125249,3.5,Action
11382417,140222,3.0,"Drama,Romance,War"
11382418,151509,3.0,"Comedy,Music,Mystery"
11382419,159109,1.5,"Comedy,Drama,Horror,Mystery"


Para calcular la probabilidad de que una película de cierto género específico supere 3.5 (el equivalente de 7 yendo de 0 a 5) utilizamos la función de probabilidad:
- $\mathbb{P}(A_g) = \frac{|A_g|}{|D_g|}$

Donde 
- $A_g$ = "película del género $g$ tiene rating mayor a 7"
- $|A_g| = $ cantidad de ratings mayores a 7 de películas del género $g$ 
- $|D_g|=$ cantidad de ratings totales del género

In [ ]:
total_ratings_by_genre = {genre:0 for genre in known_genres}
positive_ratings_by_genre = {genre:0 for genre in known_genres}
for genre in known_genres:
  total_ratings_by_genre[genre] = np.sum(df_ratings_and_genres["genres"].str.contains(genre))
  positive_ratings_by_genre[genre] = np.sum((df_ratings_and_genres["genres"].str.contains(genre)) & 
                                            (df_ratings_and_genres["rating"] > 3.5))

In [140]:
positive_ratings_by_genre

{'Action': 3319068,
 'Adventure': 3319068,
 'Animation': 3319068,
 'Comedy': 3319068,
 'Crime': 3319068,
 'Documentary': 3319068,
 'Drama': 3319068,
 'Family': 3319068,
 'Fantasy': 3319068,
 'Foreign': 3319068,
 'History': 3319068,
 'Horror': 3319068,
 'Music': 3319068,
 'Mystery': 3319068,
 'Romance': 3319068,
 'Science Fiction': 3319068,
 'TV Movie': 3319068,
 'Thriller': 3319068,
 'War': 3319068,
 'Western': 3319068}

In [124]:
total_ratings_by_genre

{'Action': 2269380,
 'Adventure': 1601072,
 'Animation': 243619,
 'Comedy': 3247720,
 'Crime': 1968948,
 'Documentary': 309666,
 'Drama': 6592158,
 'Family': 581271,
 'Fantasy': 860871,
 'Foreign': 151722,
 'History': 451187,
 'Horror': 925560,
 'Music': 403901,
 'Mystery': 1170306,
 'Romance': 2104830,
 'Science Fiction': 1388482,
 'TV Movie': 74260,
 'Thriller': 2894106,
 'War': 242783,
 'Western': 259836}

### N7


-  Dado el ROI de las películas, para aquellas cuyo ROI (Return of Investment) esté entre -10 y 10. ¿Cúal es el top 10 palabras que aparecen más de 5 veces y no son stopwords que generan mayor ROI promedio según su aparición en las descripciones?

### C5

- ¿Cúal es la película cuya entropía de las ratings que recibe es más alta? Esta será la película con opiniones más variadas. Considere sólo películas con más de 30 ratings.

### V8

- Utilizando la tabla de imdb_actors para estimar los géneros, realice la siguiente visualización. La misma debe tener el mismo estilo y rangos pero puede variar ligeramente debido a supuestos al aproximar valores. 

### V6

- Consiga la probabilidad de que una productora haga una película según el género para cada uno de los géneros. Para las productoras con participación total mayor a 100 en géneros, ¿cuáles son las dos productoras con más distancia coseno respecto de sus probabilidades? Realice este plot reemplazando los nombres de las productoras por los que corresponden 

## Parte II: Visualización de datos